In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import sys

In [9]:
def get_links(result_link):
    """Gets all article links on a given page on a news site"""
    links=[]
    r = requests.get(result_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    links_results = soup.findAll('h2', {"class":"entry-title"}) #Gets article link wrappers
    
    for result in links_results:
        links.append(result.find('a').get('href'))

    return links

def get_text(article_link):
    """Takes in a link to an article and returns the formatted article as text"""
    article=""
    r = requests.get(article_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    article_body = soup.find(class_='entry')

#     print(article_body.findAll('p'))
    for p in article_body.findAll('p'):
        excerpt = p.text.replace(u'\xa0', u' ')
        article = article + " " + excerpt
        
    return article

In [10]:
articles=[]
MAX_PAGES = 21

# get_links('http://www.globalclimatescam.com/page/2/')

for i in range(1, MAX_PAGES):
    sys.stdout.write("\rPage: {}, total articles: {}".format(i, len(articles)))
    sys.stdout.flush()
    
    links = get_links('http://www.globalclimatescam.com/page/{}/'.format(i))
    
    for link in links:
        articles.append(get_text(link))    

Page: 20, total articles: 285

In [11]:
articles2 = list(filter(None, articles)) 
df = pd.DataFrame(articles2, columns=['article']).drop_duplicates(keep='first')
df['denial?'] = [1]*len(df.index)
print('Num. articles: {}'.format(len(df)))
df.head(5)

Num. articles: 297


,article,denial?
0,In honor of Al Gore’s new movie Elmer and th...,1
1,By Elmer Beauregard 1. The Globe Isn’t Warmi...,1
2,"Dr. Judith Curry\nPresident, Climate Forecas...",1
3,CNBC.com\n Environmental Protection Agency Ad...,1
4,By Elmer Beauregard I decided to put part of...,1


54     Reposted from The Fabius Maximus Blog Larry K...
55     Guest geology/geophysics drive-by by David Mi...
56     Guest Post by Willis Eschenbach There’s an ol...
57     Guest essay by Eric Worrall h/t Dr. Willie So...
Name: article, dtype: object

In [101]:
df.to_csv('../data/data_wattsup.csv', index=False)